In [1]:
import pandas as pd
import numpy as np

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import treebank
from nltk.tag.sequential import ClassifierBasedPOSTagger
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('treebank')
stop_words = set(stopwords.words('english'))
stop_words.update( ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])

train_sents = treebank.tagged_sents()
tagger = ClassifierBasedPOSTagger(train=train_sents)
stemmer = SnowballStemmer('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asim.tewari\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asim.tewari\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\asim.tewari\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


In [2]:
new_df = pd.read_csv('liwc_final_question_type.csv')

In [3]:
import random
random.seed(0)
samples= list(new_df.person_id.unique())
num_samples = len(samples)
train_ids = random.sample(samples,round(0.8*num_samples))
test_ids = list(set(samples)-set(train_ids))

df_train=pd.DataFrame()
for i in train_ids:
  df_train=df_train.append(new_df[new_df['person_id']==i])

df_test=pd.DataFrame()
for i in test_ids:
  df_test=df_test.append(new_df[new_df['person_id']==i])

X_train = df_train.drop('y',axis=1)
X_test = df_test.drop('y',axis=1)
y_train = df_train['y']
y_test = df_test['y']

In [5]:
X_test = X_test.reset_index()
X_train = X_train.reset_index()
X_train.drop(['index'],inplace=True,axis=1)
X_test.drop(['index'],inplace=True,axis=1)
X_train.drop('person_id', axis=1, inplace=True)
X_test.drop('person_id', axis=1, inplace=True)
X_train.drop('question', axis=1, inplace=True)
X_test.drop('question', axis=1, inplace=True)

In [6]:
def get_sentence(sentens_edna):
  index = 0
  # corpus = []
  stemmed_words = list()
  # for sentens_edna in IN_x:
  word_tokens1 = [i for i in word_tokenize(sentens_edna) if i not in stop_words]
  sentens_in = ' '.join(word_tokens1)
#   for w in sentens_in.split():
#     stemmed_words.append(stemmer.stem(w))
#   sentence = ' '.join(stemmed_words)
  # corpus.append(sentence)
  return sentens_in


In [10]:
X_train['sentence'] = X_train['Source (D)'].apply(get_sentence)
X_test['sentence'] = X_test['Source (D)'].apply(get_sentence)

In [11]:
new_df['sentence'] = new_df['Source (D)'].apply(get_sentence)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [14]:
X_data = X_train['sentence'].copy()
y_data = y_train.values


print("Extracting features from the training data using a sparse vectorizer")
# t0 = time()
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                              stop_words='english',ngram_range=(1, 3))
vectorizer.fit(X_data)
# duration = time() - t0
feature_names = vectorizer.get_feature_names()

# if opts.select_chi2:
# print("Extracting %d best features by a chi-squared test" %    opts.select_chi2)
# t0 = time()
# ch2 = SelectKBest(chi2, k=opts.select_chi2)
# X_train = ch2.fit_transform(X_train, y_train)
# X_test = ch2.transform(X_test)
# if feature_names:
#     # keep selected feature names
#     feature_names = [feature_names[i] for i
#                       in ch2.get_support(indices=True)]
# print("done in %fs" % (time() - t0))
# print()

# if feature_names:
#     feature_names = np.asarray(feature_names)
print(feature_names)

def trim(s):
    """Trim string to fit on terminal (assuming 80-column display)"""
    return s if len(s) <= 80 else s[:77] + "..."

Extracting features from the training data using a sparse vectorizer
['00', '00 00', '00 00 work', '00 000', '00 000 subscribers', '00 night', '00 night iterations', '00 standard', '00 standard ve', '00 work', '00 work schedule', '000', '000 subscribers', '000 subscribers youtube', '10', '10 10', '10 10 help', '10 cheating', '10 cheating mean', '10 got', '10 got interested', '10 help', '10 help starting', '10 million', '10 million downloads', '10 portable', '10 portable start', '10 scheduled', '10 scheduled earlier', '10 standard', '10 standard ve', '10 stored', '10 stored 50', '10 yards', '10 yards lot', '10 years', '10 years line', '10 years work', '100', '100 coding', '100 coding zero', '100 pages', '100 pages good', '1082', '1082 damascus', '1082 damascus challenge', '10s', '10s millions', '10s millions people', '10th', '10th 11th', '10th 11th completed', '10th used', '10th used solve', '10th work', '10th work like', '11', '11 10', '11 10 got', '111', '111 born', '111 born word', '

In [22]:
new_features=vectorizer.transform(new_df['sentence'])


In [26]:
new_features2 = pd.DataFrame(new_features.toarray(), columns = feature_names)

In [28]:
new_features2.to_csv('tfidf_features_full.csv',index=False)